# **Install Dependencies**

1. Huggingface libraries to use Mistral 7B LLM (Open Source LLM)

2. LangChain library to call LLM to generate reposne based on the prompt

In [1]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

#LangChain related libraries
!pip install -q -U langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00


Import required Libraries and check whether we have access to GPU or not.

In [2]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4



# **Using Mistral 7 Billion parameter LLM (Open source from HuggingFace)**

Mistral 7B LLM, although big in size (approximately 14 GB), presents a challenge when loading on a Colab notebook, requiring larger GPUs and making it incompatible with the free T4 GPU.

To address this issue, we employ a sharded version of the Mistral LLM. This version is loaded in smaller increments (approximately 1.9 GB each) in the notebook and subsequently aggregated into a single file. This allows us to seamlessly utilize the LLM on a free Colab GPU (T4 GPU) without incurring any costs.

1. BitsAndBytesConfig:

  - load_in_4bit
  - bnb_4bit_use_double_quant
  - bnb_4bit_quant_type
  - bnb_4bit_compute_dtype

2. AutoModelForCausalLM.from_pretrained

3. AutoTokenizer.from_pretrained


In [7]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

# **Creating pipelines to run LLM at Colab notebook**

It uses

1. Huggingface transformers pipeline
  - This is an object that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

  - We are using "text-generation" task. It's sort of chatGPT where you ask question and model respond with answer of that question based on it's intelligence.


2. LangChain HuggingFacePipeline
  - Integrating "transformers.pipeline" with LangChain.
  - We will be using LangChain extensively later and this is first step to integrate our LLM with LangChain


In [8]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
    temperature = 0.5,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# prompt and response

In [9]:
text = "What is the purpose of life?"
response = mistral_llm.invoke(text)
print(response)


A: To be happy.


In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#### Prompt Template
prompt_template = """<s>[INST] You are a reliable and trustworthy assistant, providing helpful and respectful responses that precisely address the context.
Answer the question below from context below :
{context}
{question} [/INST] </s>
"""

question = """When did Virat Kohli first win?"""
context = """ Virat Kohli was born on November 5, 1988, in Delhi, India. He grew up in Delhi and was one of the first to train at the West Delhi Cricket Academy, created in 1998. In 2002 he played for the Delhi Under-15 team and was the highest run scorer in the 2003–04 Vijay Merchant Trophy, playing for the Delhi Under-17 team.

In February 2006 Kohli made his domestic debut for Delhi in a one-day match against Services (a team representing the Indian armed forces) but did not get a chance to bat. He scored only 10 runs in his first-class debut (first-class cricket refers to matches that last three or more days and feature two sides of 11 players each) against Tamil Nadu in November that year. He scored 90 runs in difficult conditions in a first-class match against Karnataka in December, helping Delhi draw the Test. In April 2007 he scored 35 runs in his T20 domestic debut against Himachal Pradesh.

Kohli captained the Under-19 Indian cricket team to victory at the ICC Under-19 World Cup in Kuala Lumpur, Malaysia, in 2008. His exploits were rewarded with an IPL contract from RCB for $30,000. He also made his international debut in an ODI that year, opening the batting and scoring 12 in a defeat of Sri Lanka in Dambulla, Sri Lanka. In 2009 he scored 405 runs in nine innings in the Emerging Players Tournament in Australia, ensuring that he would be at the top of the national team selectors’ minds."""

prompt = PromptTemplate(template=prompt_template, input_variables=["question","context"])

llm_chain = LLMChain(prompt=prompt, llm=mistral_llm)

response = llm_chain.invoke({"question":question,"context":context})

print(response['text'])


Virat Kohli first won when he led the Under-19 Indian cricket team to victory at the ICC Under-19 World Cup in Kuala Lumpur, Malaysia, in 2008.
